# CUnumeric-based Navier-Stokes solver

In this hands-on demo, we will implement a simple Navier Stokes
finite-difference solver based on CUnumeric. Since CUnumeric is a drop-in
replacement for Numpy, this is (supposed to be) very easy! We will also couple
our CFD simulations with some simple Machine Learning tasks (e.g. PCA for data
compression). We will also discuss the performance of our application using
tools provided by Legate.

1. [Introduction](#introduction) (What are Cunumeric and Legate? + useful resources)
2. [Getting started](#getting-started) (How to install and run Cunumeric)
3. [A simple Navier-Stokes solver](#a-simple-navier-stokes-solver)
4. [Performance](#performance-evaluation) (evaluation, profiling)
5. [A better implementation](#a-better-implementation) (?)
6. [Add some Machine Learning](#add-some-machine-learning)
7. [Use of heterogeneous resources](#use-heterogeneous-resources) (how to dispatch computations to GPUs/CPUs)

For any question, feel free to contact me at [cl2292@stanford.edu](mailto:cl2292@stanford.edu), or directly
reach out to the [Legate](https://github.com/nv-legate/legate.core) or
[CUnumeric](https://github.com/nv-legate/cunumeric) developers. If you need
help (e.g. API coverage, best practice, etc...), the most useful resource is the [Cunumeric documentation](https://nv-legate.github.io/cunumeric/23.11/).

## Introduction

See [slides](../slides/Cunumeric_slides_demo.pptx).

## Getting started

Note: Since CUnumeric is a drop-in replacement for Numpy, you can install first
write your application in vanilla Numpy, and then install CUnumeric later.

First case (the easy one), if you have a linux-64 system, and have either an
Nvidia GPU Volta or later (or if you do not intend to use GPU support), then
Cunumeric and Legate are available as a Conda package:
```console
conda install -c nvidia -c conda-forge -c legate cunumeric
```

Otherwise, you need to first build Legate from source, based on [this
guide](https://github.com/nv-legate/legate.core/blob/branch-24.01/BUILD.md). It
is strongly advised to build Legate in a conda environment.
```console
git clone https://github.com/nv-legate/legate.core.git
```
Legate provides a script to help you set-up a conda environment (replace the
python version and OS with appropriate values, run the script with `--help` to
see all available options):
```console
./scripts/generate-conda-envs.py --python 3.10 --os osx
```
After installing the dependencies, Legate can be built with:
```console
./install.py --max-dim 4 --openmp --clean
```

After completing the installation of Legate, you can clone and build Cunumeric:

```console
git clone https://github.com/nv-legate/cunumeric.git
```

then:

```console
./install.py --max-dim 4 --openmp --clean
```

## A simple Navier-Stokes solver

We will implement a solver that solver integrates the three-dimensional, compressible, single-component Navier-Stokes equations
for an ideal gas. These equations and numerical integration are detailed below.


![Navier Stokes equations](../images/NS_equation_1.png)


In [1]:
# Import Cunumeric instead of numpy
import cunumeric as num

# Define some constants
constants = {
    "grid": (128,) * 3,
    "rho_0": 1.0,
    "p0": 1e5,
    "mu": 0.0,
}

# Create variables
rho = num.full(constants["grid"], constants["rho_0"])
rhou = num.zeros(constants["grid"])
rhov = num.zeros(constants["grid"])
rhow = num.zeros(constants["grid"])
rhoE = num.zeros(constants["grid"])

[0 - 100c88580]    0.000503 {4}{threads}: reservation ('CPU proc 1d00000000000003') cannot be satisfied


### Convection fluxes

First let's implement a function that computes the convection fluxes for the 5
flow variables.

![Convection fluxes](../images/Convection_fluxes.png)

In [1]:
def convection(rho, rhou, rhov, rhow, rhoE):
    

[0 - 1007ec580]    0.000067 {4}{threads}: reservation ('CPU proc 1d00000000000003') cannot be satisfied


## Performance evaluation

## A better implementation

## Add some Machine Learning

## Use heterogeneous resources